In [40]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 292 kB/s eta 0:00:01
     |████████████████████████████████| 116 kB 3.2 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.1.0-py3-none-any.whl size=106206 sha256=30f476685666ff93103853693def61a5f2db3528b8725421d9abec4b3c7afe01
  Stored in directory: /Users/JamesMcFadden/Library/Caches/pip/wheels/12/af/e0/2adc6007eb052d6eb4ac8f01c5f699e4bc479f5ba764aadeb6
Successfully built smart-open


In [120]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     /Users/JamesMcFadden/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [119]:
from tqdm import tqdm
import pandas as pd
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import re
from nltk.corpus import wordnet
from nltk.corpus import words
import nltk

In [89]:
train = pd.read_csv('train_quotes.csv')

In [90]:
train = train.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [91]:
train.head()

,Quote,Likes
0,"She turned to look at him, and he was already ...",210
1,Being in a religion is important but you must ...,0
2,Shortcuts will only satisfy and give you succe...,0
3,Though the earth contains greater energy and m...,5
4,A novel is a mirror walking along a main road.,130


In [92]:
train.shape

(56617, 2)

In [93]:
quotes = train['Quote'].tolist()

In [94]:
# Preprocessing - to remove length-1 words, and remove non-alphabet symbols
def preprocessing(quotes):

    processed_quotes = []
    
    for quote in tqdm(quotes):
        
        # remove other non-alphabets symbols with space (i.e. keep only alphabets and whitespaces).
        processed = re.sub('[^a-zA-Z ]', '', quote)
        
        words = processed.split()
        
        # keep words that have length of more than 1 (e.g. gb, bb), remove those with length 1.
        processed_quotes.append(' '.join([word for word in words if len(word) > 1]))
    
    return processed_quotes

In [95]:
quotes = [x for i,x in enumerate(quotes) if i!=38467]

In [96]:
quotes_processed = preprocessing(quotes)

100%|██████████| 56616/56616 [00:00<00:00, 73661.48it/s]


In [97]:
def preprocessing_2(quotes):
    
    processed_quotes = []

    for quote in tqdm(quotes):
        tokens = word_tokenize(quote)
        # Convert to lower case
        tokens = [w.lower() for w in tokens]
        # Remove punctuation
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # Remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        # Filter out stopwords
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        
        processed_quotes.append(words)
        
    return processed_quotes

In [98]:
quotes_processed_final = preprocessing_2(quotes_processed)

100%|██████████| 56616/56616 [00:25<00:00, 2228.64it/s]


In [130]:
len(quotes_processed_final)

55869

In [131]:
# Words = list(set(words.words()))

In [132]:
for i in range(len(quotes_processed_final)):
    for j in range(len(quotes_processed_final[i])):
        if not wordnet.synsets(quotes_processed_final[i][j]):#Comparing if word is non-English
            quotes_processed_final.pop(i)

IndexError: list index out of range

# Training Word2Vec Model

In [19]:
model = Word2Vec(sentences = quotes_processed_final, size = 100, min_count = 1, window = 5, workers = 4)

In [20]:
# Vocab size
words = list(model.wv.vocab)
print('Vocabulary size: {}'.format(len(words)))

Vocabulary size: 89689


In [21]:
# Testing the model
# Doing math on the word vectors
model.wv.most_similar_cosmul('life')

[('living', 0.9606923460960388),
 ('lives', 0.9599635004997253),
 ('journey', 0.9569297432899475),
 ('everyday', 0.9508804678916931),
 ('regrets', 0.9503328204154968),
 ('moments', 0.9495221972465515),
 ('fullest', 0.9493431448936462),
 ('meaningful', 0.9480739831924438),
 ('part', 0.9455075263977051),
 ('lifetime', 0.9453042149543762)]

In [22]:
# Save model
filename = 'word2vec_train.txt'
model.wv.save_word2vec_format(filename, binary = False)